In [1]:
import time
from datetime import timedelta
import html

import pandas as pd
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Read csv files Java Answers

In [2]:
df = dd.read_csv('spotbugxmlreports_csv/spotbugReport*.csv', engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)


#### Explore dataframe

In [3]:
## explore one patition
#one_pat_df = df.partitions[1].compute()

In [4]:
df

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,endBytecode,isStatic,longMsg,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
npartitions=577,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Start a Dask cluster using SLURM jobs as workers

In [5]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target" : False,  # avoid spilling to disk
        "distributed.worker.memory.spill" : False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

Spawn between 20 to 100 workers and connect a client to be able use them.

In [6]:
#cluster.scale(n=20) # ask for 20 jobs or workers
# This also works with adaptive clusters. This automatically launches and kill workers based on load.
# we tell our cluster to autoscale between 10 and 20 workers depending on the load
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
#cluster.adapt(maximum_jobs=20)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.166:41607 Dashboard: http://192.168.94.166:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
df_init_len = df.shape[0].compute()
print('Initial number of volations {}'.format(df_init_len))

Initial number of volations 534978


In [8]:
df.head()

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,...,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
0,0,ce90bd5e82bd4d0d11cd96245d7fcadf,18,UrF,PERFORMANCE,2,URF_UNREAD_FIELD,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Unread field,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Code_10203353_10202514_4576_2,NaN,...,NaN,NaN,true,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Code_10203353_10202514_4576_2,2,...,NaN,NaN,NaN,NaN,NaN,Code_10203353_10202514_4576_2.java,Code_10203353_10202514_4576_2.java,1,NaN,NaN


In [9]:
df.tail()

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,...,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Code_997269_332079_2894_1,NaN,...,NaN,NaN,NaN,NaN,NaN,Code_997269_332079_2894_1.java,Code_997269_332079_2894_1.java,NaN,NaN,NaN
801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,In Code_997269_332079_2894_1.java,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Field Code_997269_332079_2894_1.hexArray,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Code_997269_332079_2894_1,2,...,NaN,NaN,true,NaN,NaN,Code_997269_332079_2894_1.java,Code_997269_332079_2894_1.java,2,1801,NaN
804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,At Code_997269_332079_2894_1.java:[line 2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
type_df = df[['type']]

In [11]:
type_df.head()

,type
0,URF_UNREAD_FIELD
1,NaN
2,NaN
3,NaN
4,NaN


In [12]:
type_df = type_df.dropna()

In [13]:
len(type_df)

41545

In [14]:
type_df.head(20)

,type
0,URF_UNREAD_FIELD
13,URF_UNREAD_FIELD
26,URF_UNREAD_FIELD
39,URF_UNREAD_FIELD
52,URF_UNREAD_FIELD
65,URF_UNREAD_FIELD
78,URF_UNREAD_FIELD
91,URF_UNREAD_FIELD
104,URF_UNREAD_FIELD
117,DM_STRING_CTOR


In [15]:
#grp_df = type_df.groupby('type').count()

In [16]:
#grp_df.head()